# High Performance Computing

## Daniel Dimitrov

## Exercise 1

Speed-up limit:
$S(p,N) < \frac{1}{f} = \frac{1}{.004} = 2 500$ 

## Exercise 2

```cpp
// my first program in C++
#include <iostream>
int main()
{	
	using namespace std;
	string name='Daniel';
	cout << "Hello World!"<<endl;
	//cin >> name;
	cout << name;
	return 0;
}
```

## Exercise 3

```cpp
#include <iostream>
#include <cmath>
int main()
{	
	using namespace std;
	float x1, x2;
	float a, b, c;
	cin >> a;
	cin >> b;	
	cin >> c;		
	x1 = (-b + sqrt(pow(b,2))-4*a*c)/(2*a);
	x2 = (-b - sqrt(pow(b,2)-4*a*c))/(2*a);		
	cout << x1<< x2<<endl;
	return 0;
}
```

## Exercise 4

```cpp
#include <iostream>

static long num_steps = 100000; 
static double steps;

int main ()
{
	int i;
	double x, pi, sum = 0.0;
	
	steps = 1.0/(double) num_steps;
	
	for (i=0; i<num_steps; i++)
	{
		x = (i+0.5)*steps;
		sum = sum + 4.0/(1.0 + x*x);
	}
	pi = steps*sum;
	std::cout<<pi<<std::endl;
	return 0;
}
```

+ Makefile: 

```
#######################################################################	
all: pi.cpp.exec
#######################################################################	

## Example 4
pi.cpp.exec : pi.cpp
	g++ pi.cpp -o pi.cpp.exec

#######################################################################	
	
clean :
	rm -f *.exec
```

+ Job Submission

```
#!/bin/bash
# a sample job submission script to submit an MPI job to the sandyb partition on Midway1

# set the job name to hello
#SBATCH --job-name=pi

# send output to pi.out
#SBATCH --output=pi.out

# receive an email when job starts, ends, and fails
#SBATCH --mail-type=BEGIN,END,DAIL

#SBATCH --account=osmlab

# this job requests 1 core. Cores can be selected from various nodes.
#SBATCH --ntasks=1

# there are many partitions on Midway1 and it is important to specify which
# partition you want to run your job on. Not having the following option, the
# sandby partition on Midway1 will be selected as the default partition
#SBATCH --partition=sandyb

# Run the process 
./pi.exe
```

## Exercise 5

```cpp
#include <iostream>
#include <cstdlib>
static int num_steps = 10000; 
static double steps;

int main ()
{
	int i, count_in=0;
	float x, y, pi;
	using namespace std;
	steps = 1.0/(double) num_steps;
	
	for (i=0; i<num_steps; i++)
	{ 
		x = static_cast<double>(rand()) / static_cast<double>(RAND_MAX);
		y = static_cast<double>(rand()) / static_cast<double>(RAND_MAX);
		
		if ((x*x + y*y) < 1)
		{
			count_in = count_in + 1; 
		}
	}
	pi = 4 * ((double) count_in / num_steps); 


	//4*count_in/<double> num_steps;
	
	cout<<count_in <<endl<<num_steps<<endl<<pi;
	return 0;
}
```

## Exercise 6

+ Asian option Monte Carlo pricing (call & put)

```cpp
/* ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ 
 * 
 * This file contains routines to serially compute the call and 
 * put price of an European option.
 * 
 * Template by Simon Scheidegger * Adapted by Daniel Dimitrov
 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++*/ 

#include <algorithm>    // Needed for the "max" function
#include <cmath>
#include <iostream>


/* ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
 A simple implementation of the Box-Muller algorithm, used to 
generate gaussian random numbers; necessary for the Monte Carlo 
method below. */

double gaussian_box_muller() {
  double x = 0.0;
  double y = 0.0;
  double euclid_sq = 0.0;

  // Continue generating two uniform random variables
  // until the square of their "euclidean distance" 
  // is less than unity
  do {
    x = 2.0 * rand() / static_cast<double>(RAND_MAX)-1;
    y = 2.0 * rand() / static_cast<double>(RAND_MAX)-1;
    euclid_sq = x*x + y*y;
  } while (euclid_sq >= 1.0);

  return x*sqrt(-2*log(euclid_sq)/euclid_sq);
}

// ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
// Pricing an Asian vanilla call option with a Monte Carlo method

double monte_carlo_call_price(const int& num_sims, const double& S, const double& K, const double& r, const double& v, const double& T) {
  double S_adjust = S * exp(T*(r-0.5*v*v));
  double S_cur = 0.0;
  double payoff_sum = 0.0;
  double A = 0.0;
  double sum_S = 0.0;

  for (int i=0; i<num_sims; i++) {
    double gauss_bm = gaussian_box_muller();
    S_cur = S_adjust * exp(sqrt(v*v*T)*gauss_bm);
	sum_S = sum_S + S_cur;
	A = sum_S / (i+1);
	
    payoff_sum += std::max(A - K, 0.0);
  }

  return (payoff_sum / static_cast<double>(num_sims)) * exp(-r*T);
}

// ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
// Pricing an Asian vanilla put option with a Monte Carlo method

double monte_carlo_put_price(const int& num_sims, const double& S, const double& K, const double& r, const double& v, const double& T) {
  double S_adjust = S * exp(T*(r-0.5*v*v));
  double S_cur = 0.0;
  double payoff_sum = 0.0;
  double A = 0.0;
  double sum_S = 0.0;
 
  

  for (int i=0; i<num_sims; i++) {
    double gauss_bm = gaussian_box_muller();
    S_cur = S_adjust * exp(sqrt(v*v*T)*gauss_bm);
	sum_S = sum_S + S_cur;
	A = sum_S / (i+1);
    payoff_sum += std::max(K - A, 0.0);
  }

  return (payoff_sum / static_cast<double>(num_sims)) * exp(-r*T);
}

// ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

int main(int argc, char **argv) {

  // Parameters                                                                             
  int num_sims = 10000000;   // Number of simulated asset paths                                                       
  double S = 100.0;  // Option price                                                                                  
  double K = 100.0;  // Strike price                                                                                  
  double r = 0.05;   // Risk-free rate (5%)                                                                           
  double v = 0.2;    // Volatility of the underlying (20%)                                                            
  double T = 1.0;    // One year until expiry                                                                         

  // Then we calculate the call/put values via Monte Carlo                                                                          
  double call = monte_carlo_call_price(num_sims, S, K, r, v, T);
  double put = monte_carlo_put_price(num_sims, S, K, r, v, T);

  // Finally we output the parameters and prices                                                                      
  std::cout << "Number of Paths: " << num_sims << std::endl;
  std::cout << "Underlying:      " << S << std::endl;
  std::cout << "Strike:          " << K << std::endl;
  std::cout << "Risk-Free Rate:  " << r << std::endl;
  std::cout << "Volatility:      " << v << std::endl;
  std::cout << "Maturity:        " << T << std::endl;

  std::cout << "Call Price:      " << call << std::endl;
  std::cout << "Put Price:       " << put << std::endl;

  return 0;
}
```